In [1]:
pip install --upgrade keras

Note: you may need to restart the kernel to use updated packages.


# Parte 1 - Construir el modelo de CNN

# Importar las librerías y paquetes

In [14]:
from keras.models import Sequential
# Existen las capas 3D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

# Inicializar la CNN

In [15]:
classifier = Sequential()

# Paso 1 - Convolución

In [16]:
# - Filters: especifica el número de filtros, es decir la dimensionalidad del espacio de salida. Se suelen usar potencias de 2 ya que están más optimizadas.
#            Se suele empezar por pocos filtros y luego ir mejorando.
# - Kernel_size especifica el tamaño de la matriz de filtros.
# - input_shape: tamaño esperado de las imágenes de entrada. Al ser imágenes de color tenemos tres dimensiones. Una matriz de 64x64 de cada color.
#                Interesa empezar con imágenes de poco tamaño y luego ir subiendo. Hay que pensar si hay que mantener tres canales o no.
# - activation: función para eliminar la linealidad del problema.
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), 
                      input_shape = (128, 128, 3), activation = "relu"))

# Paso 2 - Max Pooling

In [17]:
# Cuanto más grande sea la matriz de pooling más información perderemos aunque ganaremos más rendimiento.
# Por defecto el stride es el tamaño de pool_size, es decir se va moviendo la ventana al completo y no hay superposición.
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Una segunda capa de convolución y max pooling

In [18]:
# Para mejorar el modelo podemos:
# 1. Añadir más capas de convolución y pooling. Una práctica común es tener tres capas de convolución y pooling: 32 en primera capa, 32 en segunda capa y más filtros en la tercera, por ejemplo 64.
# 2. Añadir más nodos a la capa totalmente conectada.
# Obtamos por la 1.
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Una tercera capa

Añadimos una tercera capa con el objetivo de mejorar la precisión de la red.

In [19]:
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Paso 3 - Flattening

In [20]:
classifier.add(Flatten())

# Paso 4 - Full Connection

In [21]:
# Estamos eligiendo 128 nodos en la capa de salida de la capa totalmente conectada. 
classifier.add(Dense(units = 128, activation = "relu"))

# Probamos a usar dropout
classifier.add(Dropout(rate = 0.5))

# Estamos utilizando 1 unidad de salida porque nuestro problema de clasificación es binario, es decir, la probabilidad de ser gato es la complementaria a ser perro o viceversa.
# Si se tienen más categorías hay que tener más neuronas de salida y utilizar softmax.
classifier.add(Dense(units = 1, activation = "sigmoid"))

# Compilar la CNN
# Como va a ser entrenada?

In [22]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])


# Parte 2 - Ajustar la CNN a las imágenes para entrenar 

In [23]:
from keras.preprocessing.image import ImageDataGenerator

# Vamos a utilizar image enhancement (o image data augmentation) para obtener un buen resultado en el conjunto de entrenamiento y testing y evitar el overfitting.
# Se cogerá una selección aleatoria del lote de imágenes y se le harán transformaciones aleatorias. Se rotarán, se escalarán...
# Con rescale 1/255 transformamos los píxeles para que vayan de 0 a 1.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# target_size: tamaño al que se escalarán las imágenes.
# batch_size: tamaño del lote de datos.
# class_mode: tenemos dos clases, perros y gatos, por lo que este método de clasificación nos vale
training_dataset = train_datagen.flow_from_directory('./dataset/training_set',
                                                    target_size=(128, 128),
                                                    batch_size=32,
                                                    class_mode='binary')

testing_dataset = test_datagen.flow_from_directory('./dataset/test_set',
                                                target_size=(128, 128),
                                                batch_size=32,
                                                class_mode='binary')

# steps_per_epoch: cuantos batches de muestras se ejecutan antes de finalizar el epoch. Colocamos el conjunto completo de imágenes que tenemos dividido por el tamaño de batch.
# validation_steps: cuantos batches de muestras se ejecutan en la validación al final de cada epoch. Colocamos el conjunto completo de imágenes de test que tenemos dividido por el tamaño de batch.
classifier.fit(training_dataset,
                        steps_per_epoch=training_dataset.n//32,
                        epochs=90,
                        validation_data=testing_dataset,
                        validation_steps=testing_dataset.n//32)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/90
250/250 [==============================] - 98s 390ms/step - loss: 0.6677 - accuracy: 0.5817 - val_loss: 0.6903 - val_accuracy: 0.5625
Epoch 2/90
250/250 [==============================] - 103s 410ms/step - loss: 0.6003 - accuracy: 0.6799 - val_loss: 0.5523 - val_accuracy: 0.7334
Epoch 3/90
250/250 [==============================] - 92s 367ms/step - loss: 0.5638 - accuracy: 0.7085 - val_loss: 0.5131 - val_accuracy: 0.7510
Epoch 4/90
250/250 [==============================] - 95s 380ms/step - loss: 0.5325 - accuracy: 0.7399 - val_loss: 0.5060 - val_accuracy: 0.7535
Epoch 5/90
250/250 [==============================] - 94s 375ms/step - loss: 0.5022 - accuracy: 0.7584 - val_loss: 0.5038 - val_accuracy: 0.7566
Epoch 6/90
250/250 [==============================] - 92s 368ms/step - loss: 0.4806 - accuracy: 0.7661 - val_loss: 0.4953 - val_accuracy: 0.7692
Epoch 7/90
250/250 [=========================

# Parte 3 - Cómo hacer nuevas predicciones

In [24]:
import numpy as np
from keras.utils import load_img, img_to_array
# leemos la imagen
test_image = load_img('./dataset/single_prediction/cat_or_dog_1.jpg', target_size = (128, 128))
# la convertimos a array
test_image_ar = img_to_array(test_image)
# expand_dims expande la forma del array. Inserta un nuevo eje que aparece en la posición especificada en axis, en la forma resultante del array expandido
# Vamos a insertar una dimensión x. Es como decir que hacemos un array con un solo elemento que es es la imagen.
# Para predecir, los modelos esperan un array
# test_image_exp = np.expand_dims(test_image_ar, axis = 0)

# en lugar de usar expand simplemente vamos a hacer un array con las dos imágenes y vamos a predecir ambas
test_image2 = load_img('./dataset/single_prediction/cat_or_dog_2.jpg', target_size = (128, 128))
test_image2 = img_to_array(test_image2)
test_images = np.array([test_image_ar, test_image2])

# realizamos la predicción
result = classifier.predict(test_images)
# Un 0 es gato, un 1 perro
print(training_dataset.class_indices)

# Imprimimos resultado
for i in range(0, test_images.shape[0]):
    # la guardamos en una variable
    prediction = result[i][0]
    if prediction == 0:
        prediction_str = 'cat'
    else:
        prediction_str = 'dog'
    # la imprimimos
    print(f"The image {i + 1} is a {prediction_str}")

1/1 [==============================] - 0s 91ms/step
{'cats': 0, 'dogs': 1}
The image 1 is a dog
The image 2 is a cat
